In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!pip install bs4
!pip install contractions

In [2]:
import pandas as pd
import numpy as np
import re
import contractions
from collections import defaultdict

from bs4 import BeautifulSoup

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

pd.options.display.max_colwidth = 500

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Read Data

In [4]:
base_path = ""

df = pd.read_csv(base_path + "amazon_reviews_us_Beauty_v1_00.tsv.gz", compression='gzip', header=0,sep='\t', quotechar='"', on_bad_lines='skip')

C:\Users\jaehw\AppData\Local\Temp\ipykernel_252508\1914733243.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base_path + "amazon_reviews_us_Beauty_v1_00.tsv.gz", compression='gzip', header=0,sep='\t', quotechar='"', on_bad_lines='skip')


## Keep Reviews and Ratings

In [5]:
parsed_df = df[["star_rating", "review_headline", "review_body"]]
parsed_df.dropna()

,star_rating,review_headline,review_body
0,5,Five Stars,"Love this, excellent sun block!!"
1,5,Thank you Alba Bontanica!,The great thing about this cream is that it doesn't smell weird like all those chemical laden ones. I get a nice healthy un-fake looking tan that isn't orange and it makes my skin soft too.
2,5,Five Stars,"Great Product, I'm 65 years old and this is all it claims to be!"
3,5,GOOD DEAL!,I use them as shower caps & conditioning caps. I like that they're in bulk. It saves a lot of money.
4,5,this soaks in quick and provides a nice base for makeup,"This is my go-to daily sunblock. It leaves no white cast at all and has a clean, pleasant scent. If you're a makeup wearer, this soaks in quick and provides a nice base for makeup. I've been using this brand for over a year. With daily use, this tube will last you a couple months."
...,...,...,...
5094302,5,Great Little Grooming Tool,"After watching my Dad struggle with his scissors to clip, what he affectionately calls his 'tuffs of ear hair'. I bought him this electric clippers...now we do we hear him mumble..and fuss about how he is certain he will cut off his ear someday.....This is a great invention...it moves at lightenind speed and clips those hairs neaty...... Great Price too!"
5094303,3,Not bad for the price,"Like most sound machines, the sounds choices are limited and most have a very noticeable cycle. The brook sound actually had a click at the end! However, the ocean and white noise had a good cycle.<br />Unfortunately, only after a year, it broke. Now, I'm looking for a better one with more sounds."
5094304,5,Best Curling Iron Ever,"I bought this product because it indicated 30 second heat up time. It is great. You plug it in, hit the on button, select a heat level (1-15) , and in less than 30 seconds it is hot. No more waiting around for the iron to heat up. Quick touch ups take no time at all. I'll never go back to the &quot;old style&quot; plug and wait."
5094305,5,"The best electric toothbrush ever, REALLY!",We have used Oral-B products for 15 years; this new model is even better. It is stronger yet thinner; generates different vibrations (3) around the toothbrush head and varies this according to pressure. Also has a built-in timer. Enjoy!


 ## We form three classes and select 20000 reviews randomly from each class.


In [7]:
class1_df = parsed_df.loc[parsed_df['star_rating'].isin([1,2])].sample(20000)
class2_df = parsed_df.loc[parsed_df['star_rating'] == 3].sample(20000)
class3_df = parsed_df.loc[parsed_df['star_rating'].isin([4,5])].sample(20000)

In [8]:
class1_df["class"] = 1
class2_df["class"] = 2
class3_df["class"] = 3

In [9]:
final_df = pd.concat([class1_df, class2_df, class3_df])

final_df['review_headline'] = final_df['review_headline'].apply(str)
final_df['review_body'] = final_df['review_body'].apply(str)

In [50]:
final_df = pd.read_pickle("./finaldf.pkl") # Use this as no constant sample (forgot)

In [51]:
final_df['review'] = final_df[['review_headline', 'review_body']].agg(' '.join, axis=1)

In [52]:
final_df = final_df.drop('star_rating', axis=1)
final_df = final_df.drop('review_headline', axis=1)
final_df = final_df.drop('review_body', axis=1)

# Data Cleaning


# Pre-processing

In [53]:
# Lowercasing
final_df["review"] = final_df["review"].str.lower()

In [54]:
# class1_df[class1_df["review_body"].str.contains("1", na=False)]
def getRidOfNonAlphabet(s):
  return re.sub(r"[^a-zA-Z]+", ' ', s)

def getRidOfHTML(s):
  return BeautifulSoup(s, "lxml").text

def getRidOfURL(s):
  return re.sub(r'http\S+', '', s)

def contractions(s):
  # Also gets rid of extra spaces
  import contractions
  ans = []
  for word in s.split():
    ans.append(contractions.fix(word))
  return ' '.join(ans)

In [55]:
final_df["review"] = final_df["review"].apply(getRidOfURL).apply(getRidOfHTML).apply(contractions).apply(getRidOfNonAlphabet)

C:\Users\jaehw\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [44]:
print("Cleaned Review Body Avg Character Count",(final_df['review'].str.len()).mean())

Cleaned Review Body Avg Character Count 303.84655


## remove the stop words 

In [45]:
def getRidOfStopWords(s):
  stopWords = set(stopwords.words('english'))
  words = word_tokenize(s)
  filteredWords = []
  for word in words:
      if word not in stopWords:
          filteredWords.append(word)
  return ' '.join(filteredWords)

In [46]:
final_df["review"] = final_df["review"].apply(getRidOfStopWords)

## perform lemmatization  

In [56]:
def lemmatize(s):
  lemmatizer = WordNetLemmatizer()
  words = word_tokenize(s)
    
  filteredWords = []
  for word in words:
      filteredWords.append(lemmatizer.lemmatize(word))
  return ' '.join(filteredWords)

def lemattizeWithDict(l):
  lemmatizer = WordNetLemmatizer()

  tags = defaultdict(lambda : wn.NOUN)
  tags['J'] = wn.ADJ
  tags['V'] = wn.VERB
  tags['R'] = wn.ADV

  words = []
  for word, t in pos_tag(l.split()):
      if word not in stopwords.words('english'):
          words.append(lemmatizer.lemmatize(word, tags[t[0]]))
  return str(' '.join(words))

In [57]:
final_df["formatted_review"] = final_df["review"].apply(lemmatize)

# TF-IDF Feature Extraction

In [1]:
# final_df = pd.read_pickle("./final_final_df.pkl")

train_x, test_x, train_y, test_y = train_test_split(final_df['formatted_review'], final_df['class'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(ngram_range=(1,3))
vectorizer.fit(final_df['formatted_review'])
train_tfidf = vectorizer.transform(train_x)
test_tfidf = vectorizer.transform(test_x)

Encoder = LabelEncoder()
train_result = Encoder.fit_transform(train_y)
test_result = Encoder.fit_transform(test_y)

NameError: name 'train_test_split' is not defined

# Perceptron

In [17]:
perceptron = Perceptron(random_state=486)
perceptron.fit(train_tfidf, train_result)
result = perceptron.predict(test_tfidf)
report = classification_report(test_result, result)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.80      0.80      3963
           1       0.72      0.72      0.72      4014
           2       0.86      0.86      0.86      4023

    accuracy                           0.79     12000
   macro avg       0.79      0.79      0.79     12000
weighted avg       0.79      0.79      0.79     12000



# SVM

In [24]:
model = svm.LinearSVC()
model.fit(train_tfidf, train_result)
result = model.predict(test_tfidf)
report = classification_report(test_result, result)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.82      0.81      3963
           1       0.75      0.71      0.73      4014
           2       0.87      0.87      0.87      4023

    accuracy                           0.80     12000
   macro avg       0.80      0.80      0.80     12000
weighted avg       0.80      0.80      0.80     12000



# Logistic Regression

In [27]:
model = LogisticRegression()
model.fit(train_tfidf, train_result)
result = model.predict(test_tfidf)
report = classification_report(test_result, result)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      3963
           1       0.73      0.72      0.73      4014
           2       0.87      0.85      0.86      4023

    accuracy                           0.80     12000
   macro avg       0.80      0.80      0.80     12000
weighted avg       0.80      0.80      0.80     12000



C:\Users\jaehw\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Naive Bayes

In [18]:
model = naive_bayes.MultinomialNB()
model.fit(train_tfidf, train_result)
result = model.predict(test_tfidf)
report = classification_report(test_result, result)
print(report)

              precision    recall  f1-score   support

           0       0.78      0.79      0.79      3963
           1       0.68      0.79      0.73      4014
           2       0.93      0.77      0.84      4023

    accuracy                           0.78     12000
   macro avg       0.80      0.78      0.79     12000
weighted avg       0.80      0.78      0.79     12000

